# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [7]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras import initializers
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_val = pd.read_csv('data/y_val.csv')
y_test = pd.read_csv('data/y_test.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

In [5]:
n_features

296

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

In [8]:
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(units = 100, activation = 'relu', input_shape = (n_features,)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(units = 50, activation = 'relu'))

# Output layer
baseline_model.add(layers.Dense(units = 1, activation = 'linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 8ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [9]:
# Numeric column names
numeric_columns = X_train_numeric.columns

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns = numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns = numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns = numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [10]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 

In [11]:
# Train the model 
normalized_input_model.fit(X_train, y_train, epochs = 150, batch_size = 32, 
                           validation_data = (X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [12]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [13]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, y_train_scaled, epochs = 150, batch_size = 32,
                    validation_data = (X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 3ms/step - loss: 0.3990 - mse: 0.3990 - val_loss: 0.1613 - val_mse: 0.1613
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2374 - mse: 0.2374 - val_loss: 0.1290 - val_mse: 0.1290
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1837 - mse: 0.1837 - val_loss: 0.1904 - val_mse: 0.1904
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1577 - mse: 0.1577 - val_loss: 0.1212 - val_mse: 0.1212
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1364 - mse: 0.1364 - val_loss: 0.1210 - val_mse: 0.1210
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1260 - mse: 0.1260 - val_loss: 0.1467 - val_mse: 0.1467
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1084 - mse: 0.1084 - val_loss: 0.1104 - val_mse: 0.1104
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1025 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0246 - mse: 0.0246 - val_loss: 0.1235 - val_mse: 0.1235
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0236 - mse: 0.0236 - val_loss: 0.1232 - val_mse: 0.1232
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0239 - mse: 0.0239 - val_loss: 0.1243 - val_mse: 0.1243
Epoch 66/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0229 - mse: 0.0229 - val_loss: 0.1251 - val_mse: 0.1251
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0226 - mse: 0.0226 - val_loss: 0.1254 - val_mse: 0.1254
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0227 - mse: 0.0227 - val_loss: 0.1222 - val_mse: 0.1222
Epoch 69/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0226 - mse: 0.0226 - val_loss: 0.1226 - val_mse: 0.1226
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0219 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0113 - mse: 0.0113 - val_loss: 0.1258 - val_mse: 0.1258
Epoch 126/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0113 - mse: 0.0113 - val_loss: 0.1274 - val_mse: 0.1274
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0111 - mse: 0.0111 - val_loss: 0.1282 - val_mse: 0.1282
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0110 - mse: 0.0110 - val_loss: 0.1261 - val_mse: 0.1261
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.1257 - val_mse: 0.1257
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0109 - mse: 0.0109 - val_loss: 0.1271 - val_mse: 0.1271
Epoch 131/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0109 - mse: 0.0109 - val_loss: 0.1269 - val_mse: 0.1269
Epoch 132/150
33/33 [==============================] - 0s 2ms/step - 

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [14]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 704us/step - loss: 0.0084 - mse: 0.0084


[0.008445678278803825, 0.008445678278803825]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [15]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 756us/step - loss: 0.1291 - mse: 0.1291


[0.12908463180065155, 0.12908463180065155]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [16]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

28233.76306087076

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'`  

In [18]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, activation = 'relu', kernel_initializer = 'he_normal', 
                          input_shape = (n_features,)))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 3ms/step - loss: 0.4604 - mse: 0.4604 - val_loss: 0.2141 - val_mse: 0.2141
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2593 - mse: 0.2593 - val_loss: 0.1680 - val_mse: 0.1680
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2140 - mse: 0.2140 - val_loss: 0.1499 - val_mse: 0.1499
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1860 - mse: 0.1860 - val_loss: 0.1401 - val_mse: 0.1401
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1713 - mse: 0.1713 - val_loss: 0.1358 - val_mse: 0.1358
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1534 - mse: 0.1534 - val_loss: 0.1277 - val_mse: 0.1277
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1397 - mse: 0.1397 - val_loss: 0.1280 - val_mse: 0.1280
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1360 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0292 - mse: 0.0292 - val_loss: 0.1383 - val_mse: 0.1383
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0288 - mse: 0.0288 - val_loss: 0.1361 - val_mse: 0.1361
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0287 - mse: 0.0287 - val_loss: 0.1349 - val_mse: 0.1349
Epoch 66/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0281 - mse: 0.0281 - val_loss: 0.1375 - val_mse: 0.1375
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0273 - mse: 0.0273 - val_loss: 0.1350 - val_mse: 0.1350
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0275 - mse: 0.0275 - val_loss: 0.1364 - val_mse: 0.1364
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0271 - mse: 0.0271 - val_loss: 0.1364 - val_mse: 0.1364
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0267 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0146 - mse: 0.0146 - val_loss: 0.1402 - val_mse: 0.1402
Epoch 126/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0144 - mse: 0.0144 - val_loss: 0.1436 - val_mse: 0.1436
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0143 - mse: 0.0143 - val_loss: 0.1414 - val_mse: 0.1414
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0140 - mse: 0.0140 - val_loss: 0.1416 - val_mse: 0.1416
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0138 - mse: 0.0138 - val_loss: 0.1428 - val_mse: 0.1428
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0137 - mse: 0.0137 - val_loss: 0.1426 - val_mse: 0.1426
Epoch 131/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0139 - mse: 0.0139 - val_loss: 0.1425 - val_mse: 0.1425
Epoch 132/150
33/33 [==============================] - 0s 2ms/step - 

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [19]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 682us/step - loss: 0.0112 - mse: 0.0112


[0.011222795583307743, 0.011222795583307743]

Evaluate the model (`he_model`) on validate data (`X_train` and `y_train_scaled`) 

In [20]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 780us/step - loss: 0.1438 - mse: 0.1438


[0.1438445746898651, 0.1438445746898651]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 

In [21]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, activation = 'relu', kernel_initializer = 'lecun_normal',
                            input_shape = (n_features, )))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.4656 - mse: 0.4656 - val_loss: 0.1826 - val_mse: 0.1826
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2228 - mse: 0.2228 - val_loss: 0.1524 - val_mse: 0.1524
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1803 - mse: 0.1803 - val_loss: 0.1366 - val_mse: 0.1366
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1676 - mse: 0.1676 - val_loss: 0.1286 - val_mse: 0.1286
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1544 - mse: 0.1544 - val_loss: 0.1293 - val_mse: 0.1293
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1410 - mse: 0.1410 - val_loss: 0.1193 - val_mse: 0.1193
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1349 - mse: 0.1349 - val_loss: 0.1206 - val_mse: 0.1206
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1235 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0239 - mse: 0.0239 - val_loss: 0.1108 - val_mse: 0.1108
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0230 - mse: 0.0230 - val_loss: 0.1119 - val_mse: 0.1119
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0229 - mse: 0.0229 - val_loss: 0.1111 - val_mse: 0.1111
Epoch 66/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0224 - mse: 0.0224 - val_loss: 0.1108 - val_mse: 0.1108
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0221 - mse: 0.0221 - val_loss: 0.1109 - val_mse: 0.1109
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0223 - mse: 0.0223 - val_loss: 0.1129 - val_mse: 0.1129
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0216 - mse: 0.0216 - val_loss: 0.1124 - val_mse: 0.1124
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0207 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0105 - mse: 0.0105 - val_loss: 0.1192 - val_mse: 0.1192
Epoch 126/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0106 - mse: 0.0106 - val_loss: 0.1192 - val_mse: 0.1192
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0105 - mse: 0.0105 - val_loss: 0.1188 - val_mse: 0.1188
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0104 - mse: 0.0104 - val_loss: 0.1194 - val_mse: 0.1194
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0103 - mse: 0.0103 - val_loss: 0.1201 - val_mse: 0.1201
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0100 - mse: 0.0100 - val_loss: 0.1200 - val_mse: 0.1200
Epoch 131/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0102 - mse: 0.0102 - val_loss: 0.1193 - val_mse: 0.1193
Epoch 132/150
33/33 [==============================] - 0s 2ms/step - 

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [22]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 699us/step - loss: 0.0078 - mse: 0.0078


[0.007768816314637661, 0.007768816314637661]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [23]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 799us/step - loss: 0.1219 - mse: 0.1219


[0.12186742573976517, 0.12186742573976517]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

In [24]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer = 'rmsprop', loss = 'mse', metrics = ['mse'])

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.3791 - mse: 0.3791 - val_loss: 0.1214 - val_mse: 0.1214
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1679 - mse: 0.1679 - val_loss: 0.0985 - val_mse: 0.0985
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1266 - mse: 0.1266 - val_loss: 0.1233 - val_mse: 0.1233
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1077 - mse: 0.1077 - val_loss: 0.1423 - val_mse: 0.1423
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0879 - mse: 0.0879 - val_loss: 0.1287 - val_mse: 0.1287
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0760 - mse: 0.0760 - val_loss: 0.1110 - val_mse: 0.1110
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0577 - mse: 0.0577 - val_loss: 0.1409 - val_mse: 0.1409
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0555 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0076 - mse: 0.0076 - val_loss: 0.1051 - val_mse: 0.1051
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0064 - mse: 0.0064 - val_loss: 0.1123 - val_mse: 0.1123
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0089 - mse: 0.0089 - val_loss: 0.0957 - val_mse: 0.0957
Epoch 66/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0094 - mse: 0.0094 - val_loss: 0.1105 - val_mse: 0.1105
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0077 - mse: 0.0077 - val_loss: 0.1158 - val_mse: 0.1158
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0077 - mse: 0.0077 - val_loss: 0.1116 - val_mse: 0.1116
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0088 - mse: 0.0088 - val_loss: 0.1015 - val_mse: 0.1015
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0066 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0049 - mse: 0.0049 - val_loss: 0.1068 - val_mse: 0.1068
Epoch 126/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0048 - mse: 0.0048 - val_loss: 0.1174 - val_mse: 0.1174
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0048 - mse: 0.0048 - val_loss: 0.1108 - val_mse: 0.1108
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0060 - mse: 0.0060 - val_loss: 0.1003 - val_mse: 0.1003
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0030 - mse: 0.0030 - val_loss: 0.1185 - val_mse: 0.1185
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0053 - mse: 0.0053 - val_loss: 0.1035 - val_mse: 0.1035
Epoch 131/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0049 - mse: 0.0049 - val_loss: 0.1202 - val_mse: 0.1202
Epoch 132/150
33/33 [==============================] - 0s 2ms/step - 

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [25]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 682us/step - loss: 0.0057 - mse: 0.0057


[0.005740173626691103, 0.005740173626691103]

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [26]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 799us/step - loss: 0.1026 - mse: 0.1026


[0.10256018489599228, 0.10256018489599228]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric  

In [27]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer = 'Adam', loss = 'mse', metrics = ['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.3423 - mse: 0.3423 - val_loss: 0.1562 - val_mse: 0.1562
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1665 - mse: 0.1665 - val_loss: 0.1424 - val_mse: 0.1424
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0960 - mse: 0.0960 - val_loss: 0.1343 - val_mse: 0.1343
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0720 - mse: 0.0720 - val_loss: 0.1362 - val_mse: 0.1362
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0495 - mse: 0.0495 - val_loss: 0.1357 - val_mse: 0.1357
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0377 - mse: 0.0377 - val_loss: 0.1430 - val_mse: 0.1430
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0357 - mse: 0.0357 - val_loss: 0.1547 - val_mse: 0.1547
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0286 - m

33/33 [==============================] - 0s 3ms/step - loss: 0.0021 - mse: 0.0021 - val_loss: 0.1252 - val_mse: 0.1252
Epoch 64/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0023 - mse: 0.0023 - val_loss: 0.1472 - val_mse: 0.1472
Epoch 65/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0024 - mse: 0.0024 - val_loss: 0.1227 - val_mse: 0.1227
Epoch 66/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0031 - mse: 0.0031 - val_loss: 0.1431 - val_mse: 0.1431
Epoch 67/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0040 - mse: 0.0040 - val_loss: 0.1217 - val_mse: 0.1217
Epoch 68/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0042 - mse: 0.0042 - val_loss: 0.1380 - val_mse: 0.1380
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0066 - mse: 0.0066 - val_loss: 0.1151 - val_mse: 0.1151
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0056 - mse: 0

33/33 [==============================] - 0s 2ms/step - loss: 0.0019 - mse: 0.0019 - val_loss: 0.1328 - val_mse: 0.1328
Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0018 - mse: 0.0018 - val_loss: 0.1270 - val_mse: 0.1270
Epoch 126/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0016 - mse: 0.0016 - val_loss: 0.1295 - val_mse: 0.1295
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0011 - mse: 0.0011 - val_loss: 0.1253 - val_mse: 0.1253
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 9.4613e-04 - mse: 9.4613e-04 - val_loss: 0.1289 - val_mse: 0.1289
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0011 - mse: 0.0011 - val_loss: 0.1284 - val_mse: 0.1284
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0012 - mse: 0.0012 - val_loss: 0.1302 - val_mse: 0.1302
Epoch 131/150
33/33 [==============================] - 0s 2ms/step - loss: 

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [28]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 810us/step - loss: 0.0018 - mse: 0.0018


[0.0018085541669279337, 0.0018085541669279337]

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [29]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 686us/step - loss: 0.1137 - mse: 0.1137


[0.11367610096931458, 0.11367610096931458]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [30]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

5/5 [==============================] - 0s 776us/step - loss: 0.2174 - mse: 0.2174


[0.21735431253910065, 0.21735431253910065]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [31]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

36636.63898373632

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.